# Example Process

### Import Libraries

In [1]:
import pandas as pd
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.rc('font', size=12)
import math
# Filter out deprecated warnings
import warnings
warnings.filterwarnings("ignore")

import time

In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


### Constant - COPY AND PASTE

In [2]:
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

### Data Import

In [3]:
data = pd.read_excel('../../data/market_data.xlsx')
data1 = data

vic_data = data[['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)', 
                 'Regions VIC Trading Total Intermittent Generation (MW)', 
                 'Regions VIC Operational Demand (MW)']]

# Since the first date is at 00:00:00, the first period should be 48
period = [48]
x = 1
while x < len(vic_data):
    for i in range(48):
        period.append(i+1)
        x += 1
        
vic_data.insert(1, 'Period', period)

vic = data[['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)', 
                 'Regions VIC Trading Total Intermittent Generation (MW)', 
                 'Regions VIC Operational Demand (MW)']]

vic.insert(1, 'Period', period)
vic.drop([0], axis=0)

vic.head(5)

,Time (UTC+10),Period,Regions VIC Trading Price ($/MWh),Regions VIC Trading Total Intermittent Generation (MW),Regions VIC Operational Demand (MW)
0,2018-01-01 00:00:00,48,90.43,146.87,4599
1,2018-01-01 00:30:00,1,92.46,131.68,4398
2,2018-01-01 01:00:00,2,87.62,119.98,4238
3,2018-01-01 01:30:00,3,73.08,123.86,4112
4,2018-01-01 02:00:00,4,70.18,132.72,3956


### Find Optimal Discharge and Charge Period - DEPENDS ON WHETHER YOUR ALGO NEEDS THIS

In [4]:
spot_price = vic.groupby(['Period'])[[PRICE]].mean()

MIN_PRICE = spot_price.min()
MAX_PRICE = spot_price.max()

max_price = 0
min_price = 1100
max_i = 0
min_i = 0

#second_max_price = 0
#second_min_price = 1000
#sec_max_loc = 0
#sec_min_loc = 0

for i in range(48-6):
    curr = spot_price.iloc[i:i+6 ,0].sum()
    
    if curr < min_price:
        #second_min_price = min_price
        min_price = curr
        #sec_min_loc = min_i
        min_i = i
        
    if curr > max_price:
        #second_max_price = max_price
        max_price = curr
        #sec_max_loc = max_i
        max_i = i
        
max_period = []
min_period = []

#sec_max_period = []
#sec_min_period = []

for i in range(6):
    max_period.append(max_i + i)
    min_period.append(min_i + i)
    
    #sec_max_period.append(sec_min_loc + i)
    #sec_min_period.append(sec_min_loc + i)
    
behaviour = []
for i in range(48):
    if i in max_period:
        behaviour.append(1)
        
    elif i in min_period:
        behaviour.append(-1)
    
    else:
        behaviour.append(0)
        
spot_price['Behaviour'] = behaviour

# the charge and discharge period are fixed in Algorithm 2, +1 to get their periods
charge_period = list(np.asarray(min_period) + 1)
discharge_period = list(np.asarray(max_period) + 1)


### Run Discharge and Charge Algorithm - CHANGE TO YOUR ALGORITHM HERE

In [5]:
def run_algo2(period, charge_period, discharge_period):
    """Run Algorithm 2"""
    raw_power = 0
    if period in charge_period:
        raw_power = -math.floor(min(POWER,(CAPACITY-opening_cap)/(CHARGE_EFF/100)*2))
    elif period in discharge_period:
        raw_power = math.floor(min(POWER,opening_cap/(DISCHARGE_EFF/100)*2))
    return raw_power

### Helper functions - COPY AND PASTE

In [6]:
def create_df(train_df):
    """ Create raw_power, dispatch, revenue, opening and closing column"""
    df = train_df[[TIME, 'Period', PRICE]]
    df['raw_power'] = np.zeros(len(train_df))
    df['dispatch'] = np.zeros(len(train_df))
    df['revenue'] = np.zeros(len(train_df))
    df['opening'] = np.zeros(len(train_df))
    df['closing'] = np.zeros(len(train_df))
    # Drop the first row. Not the first period
    df = df.drop([0], axis=0)
    df['revenue'] = np.nan

    return df

def get_opencap(index, df):
    """Get the opening battery capacity per index """
    if index != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    return opening_cap

def get_dispatch(rawPower):
    """ Get the power dispatched """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch
    
def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity"""
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue"""
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = math.ceil(price * dispatch * factor)
    return revenue

def show_result(df):
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))

### Run the entire optimization algorithm

In [7]:
start = time.time()

df = create_df(vic)
    
for i in list(df.index):
    period = df.at[i, "Period"]
    price = df.at[i, PRICE]

    # Battery Opening Capacity
    opening_cap = get_opencap(i, df)

    # Raw Power - INSERT YOUR ALGORITHM HERE
    rawPower = run_algo2(period, charge_period, discharge_period)
    df.at[i, "raw_power"] = rawPower

    # Market Dispatch
    dispatch = get_dispatch(rawPower)
    df.at[i,"dispatch"] = dispatch

    # Closing Capacity
    df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

    # Market Revenue
    df.at[i,"revenue"] = get_revenue(price, dispatch) 

end = time.time()
print("Time Complexity for running: {time_taken}s".format(time_taken = end-start))

Time Complexity for running: 4.655720949172974s


In [8]:
show_result(df)

Total revenue in the dataset: 55937605.0
Total days in the dataset: 1322.0
Revenue per day: 42312.86308623298
